In [1]:
%matplotlib inline

# Guardar y cargar el modelo

En esta sección veremos cómo mantener el estado del modelo guardando, cargando y ejecutando predicciones del modelo.

In [2]:
import torch
import torch.onnx as onnx
import torchvision.models as models

## Guardar y cargar pesos de modelos

Los modelos de PyTorch almacenan los parámetros aprendidos en un diccionario de estado interno, llamado ``state_dict``. Estos se pueden conservar a través del método ``torch.save``:

In [3]:
model = models.vgg16(pretrained=True)
torch.save(model.state_dict(), 'model_weights.pth')

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\mfnunez/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100%|██████████| 528M/528M [07:08<00:00, 1.29MB/s]


Para cargar pesos de modelo, primero debe crear una instancia del mismo modelo y luego cargar los parámetros usando el método ``load_state_dict()``.

In [4]:
model = models.vgg16() # we do not specify pretrained=True, i.e. do not load default weights
model.load_state_dict(torch.load('model_weights.pth'))
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

 > **Nota**:
 > asegúrese de llamar al método ``model.eval()`` antes de hacer inferencias para configurar las capas de normalización por lotes y de abandono en el modo de evaluación. No hacer esto producirá resultados de inferencia inconsistentes.

## Guardar y cargar modelos con formas

Al cargar pesos de modelo, necesitábamos crear una instancia de la clase de modelo primero, porque la clase define la estructura de una red. Es posible que deseemos guardar la estructura de esta clase junto con el modelo, en cuyo caso podemos pasar ``model`` (y no ''model.state_dict()'') a la función de guardar:

In [5]:
torch.save(model, 'model.pth')

Luego podemos cargar el modelo así:



In [6]:
model = torch.load('model.pth')

 > **Nota**:
 > Este enfoque utiliza el módulo [pickle](https://docs.python.org/3/library/pickle.html) de Python al serializar el modelo, por lo que se basa en la definición de clase real que estará disponible al cargar el modelo.

## Exportar modelo a ONNX

PyTorch también tiene soporte de exportación ONNX nativo. Sin embargo, dada la naturaleza dinámica del gráfico de ejecución de PyTorch, el proceso de exportación debe atravesar el gráfico de ejecución para producir un modelo ONNX persistente. Por esta razón, se debe pasar una variable de prueba del tamaño apropiado a la rutina de exportación (en nuestro caso, crearemos un tensor cero ficticio del tamaño correcto):

In [7]:
input_image = torch.zeros((1,3,224,224))
onnx.export(model, input_image, 'model.onnx')

C:\Users\mfnunez\Anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Hay muchas cosas que puede hacer con el modelo ONNX, incluida la ejecución de inferencias en diferentes plataformas y en diferentes lenguajes de programación. Para obtener más detalles, recomendamos visitar el [tutorial de ONNX](https://github.com/onnx/tutorials).